In [ ]:
# Clean up!
import sqlite3

DRIFT_DB_PATH = "..\\data\\driftDb.db"

con = sqlite3.connect(DRIFT_DB_PATH)
cursor = con.cursor()

# list all tables
def list_all_tables():
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    cursor.execute(sql_query)
    tables = cursor.fetchall()
    print("Table count: ".format(len(tables)))
    print("Tables: \n".format(tables))
  
TABLE_PARAMETER = "{TABLE_PARAMETER}"
DROP_TABLE_SQL = f"DROP TABLE {TABLE_PARAMETER};"
GET_TABLES_SQL = "SELECT name FROM sqlite_schema WHERE type='table';"

def delete_all_tables(con):
    tables = get_tables(con)
    delete_tables(con, tables)

def get_tables(con):
    cursor = con.cursor()
    cursor.execute(GET_TABLES_SQL)
    tables = cursor.fetchall()
    cursor.close()
    return tables

def delete_tables(con, tables):
    cursor = con.cursor()
    for table, in tables:
        sql = DROP_TABLE_SQL.replace(TABLE_PARAMETER, table)
        cursor.execute(sql)
    cursor.close()

list_all_tables()  
delete_all_tables(con)    
list_all_tables()


In [ ]:
import weaviate

print("Creating Weaviate client")
client = weaviate.Client("http://localhost:8080")
print("Client created")

print("Deleting all schemas")
client.schema.delete_all()
print("All schemas deleted.")

In [ ]:
# Create the table for benchmark cosine similarity results


# SQLite benchmark sample and completion functions
# user
# insert_benchmark("chat_id00000000", "bmk_prompt", "bmk_completion")
# insert_api_completion("chat_id00000000", "bmk_prompt", "bmk_completion", 0.987654321, "api_completion")

run_time = ct = datetime.datetime.now().replace(second=0, microsecond=0)

# inserts default value of 1.0 for cos_sim
def insert_benchmark(chat_id, bmk_prompt, bmk_completion):
    sql = "INSERT INTO tSample (chat_id, run_time, bmk_prompt, bmk_completion, cos_sim) VALUES ('{}', '{}', \"{}\", \"{}\", {});".format(chat_id, run_time, bmk_prompt, bmk_completion, 1.0);
    print(sql)
    sql_cursor.execute(sql)
    sql_conxn.commit()
    

def insert_api_completion(chat_id, bmk_prompt, bmk_completion, cos_sim, api_completion):
    sql = "INSERT INTO tSample (chat_id, run_time, bmk_prompt, bmk_completion, cos_sim, api_completion) VALUES ('{}', '{}', \"{}\", \"{}\", {}, \"{}\");".format(chat_id, run_time, bmk_prompt, bmk_completion, cos_sim, api_completion);
    print(sql)
    sql_cursor.execute(sql)
    sql_conxn.commit()    


try:
    sql_conxn = sqlite3.connect(DRIFT_DB_PATH)
    sql_cursor = sql_conxn.cursor()
    #sql_cursor.execute(""" PRAGMA foreign_keys = 0;""")
    # sql_cursor.execute(""" DELETE FROM tSample;""")
    # sql_cursor.execute(""" DROP TABLE tSample;""")
    sql_cursor.execute(""" CREATE TABLE tSample (
          chat_id NOT NULL
        , run_time REAL (64) NOT NULL DEFAULT ((datetime('now', 'localtime')))
        , bmk_prompt TEXT NOT NULL
        , bmk_completion TEXT NOT NULL
        , cos_sim NUMERIC NOT NULL
        , api_completion TEXT );""")

    # sql_cursor.execute(""" PRAGMA foreign_keys = 1; """)
    res = sql_cursor.execute("SELECT name FROM sqlite_master")
    res.fetchone()


    insert_benchmark("chat_id00000000", "bmk_prompt", "bmk_completion")
    insert_api_completion("chat_id00000000", "bmk_prompt", "bmk_completion", 0.987654321, "api_completion")

    res = sql_cursor.execute("SELECT * FROM tSample;")
    rows = res.fetchall()
    print(json.dumps(rows, indent=4))

    sql_cursor.execute("DELETE FROM tSample;")
    sql_conxn.commit()

    print("sqlite3 script completed.")
    
except sqlite3.Error as error:
    print("sqlite3.Error: ", error)
finally:
    if sql_conxn:
        sql_cursor.close()
        sql_conxn.close()   